# Import Package 

In [2]:
import re
import json
import math
import numpy as np
import collections
import xml.dom.minidom
from stemming.porter2 import stem

#  Input from .txt File into dictionary

In [4]:
# Term -> (Document -> Position)
# input term_doc_pos_dict
# {'Term1': {doc1: [position1, position2], doc2: [position1, position2]}
#  'Term2': {doc1: [position1, position2, position3], doc2: [position1, position2]}}

file = open('index_ts.txt', 'r') 
js = file.read()
term_doc_pos_dict = json.loads(js) 
file.close() 

# term_doc_pos_dict

In [5]:
# Term -> Position
# input from term_position_dict
# {'Term1': [[doc1, pos1], [doc2, pos2], [doc3, pos3]
#  'Term2': [[doc1, pos1], [doc2, pos2], [doc3, pos3]}

file = open('index_ts_pos.txt', 'r') 
js = file.read()
term_position_dict = json.loads(js) 
file.close() 

# term_position_dict

# Get Score of each Query

In [6]:
# stopping words list
with open('englishST.txt', 'r') as eng:
    eng_str = eng.read()
eng_list = eng_str.split()  

def takeSecond(elem):
    return elem[1]

def getScore(s, term_doc_pos_dict):
    s_list = s.split()
    for m in range(len(s_list)):
        s_list[m] = s_list[m].lower()  # lower()

    # Stopping 
    stop_list = []
    for i in s_list:
        if (i not in eng_list):
            stop_list.extend([i])
    
    # Normalisation
    w_list = []
    for j in stop_list:
        w_list.extend([stem(j)])
    
    # get document list
    doc_list = list(term_doc_pos_dict[w_list[0]].keys()) 
    for i in range(len(w_list)-1):
        doc_list = list(set(doc_list).union(term_doc_pos_dict[w_list[i+1]].keys())) 
    for j in range(len(doc_list)):
        doc_list[j] = int(doc_list[j])
    doc_list.sort()
    
    # get score list
    score_list = []
    for i in range(len(doc_list)):
        score_doc = 0
        for j in range(len(w_list)):           
            if (str(doc_list[i]) in list(term_doc_pos_dict[w_list[j]].keys())):
                a = 1 + math.log10(len(term_doc_pos_dict[w_list[j]][str(doc_list[i])]))
                # print(a)
                b = math.log10(1000/len(term_doc_pos_dict[w_list[j]].keys()))
                # print(b)
                score_doc = score_doc + a * b
        score_list.extend([[int(doc_list[i]), format(score_doc, '.4f')]])  
    
    # sort according to 1st column
    score_list.sort(key=takeSecond, reverse=True)
              
    return score_list

In [7]:
getScore("income tax reduction", term_doc_pos_dict)

[[65, '4.8040'],
 [3533, '4.7264'],
 [3562, '3.5454'],
 [3608, '3.4910'],
 [141, '3.3262'],
 [361, '3.3262'],
 [92, '3.2311'],
 [3829, '3.1818'],
 [3420, '3.1273'],
 [3734, '3.0561'],
 [3387, '2.9626'],
 [3599, '2.9626'],
 [3529, '2.9569'],
 [3405, '2.9026'],
 [3706, '2.7539'],
 [3403, '2.6467'],
 [125, '2.5980'],
 [3417, '2.5980'],
 [3596, '2.5541'],
 [177, '2.5185'],
 [3503, '2.5024'],
 [282, '2.4832'],
 [3818, '2.4728'],
 [163, '2.4116'],
 [3708, '2.3733'],
 [3699, '2.3216'],
 [3495, '2.2833'],
 [3710, '2.2833'],
 [184, '2.2832'],
 [3838, '2.2832'],
 [3519, '2.2450'],
 [3817, '2.2450'],
 [3345, '2.2345'],
 [270, '2.1550'],
 [3407, '2.1550'],
 [3680, '2.1550'],
 [3890, '2.1550'],
 [314, '2.1025'],
 [3535, '2.1025'],
 [3606, '2.0641'],
 [3327, '1.9742'],
 [3441, '1.9742'],
 [3590, '1.9742'],
 [3705, '1.9742'],
 [3834, '1.9742'],
 [33, '1.9358'],
 [170, '1.9358'],
 [268, '1.9358'],
 [3337, '1.9358'],
 [3622, '1.9358'],
 [3775, '1.9358'],
 [3827, '1.9358'],
 [3700, '1.8259'],
 [352, '1.

# Generate term_score_dict 

In [9]:
# Input query txt
with open('queries.lab3.txt', 'r') as f:
    line_list = f.readlines()
f.close()

term_score_dict = {}
num_list = []
l_list = []
r = """[0-9]"""

# get the number
for i in range(len(line_list)):  
    num_list.extend([line_list[i].split()])  # split 
print(num_list)
# get the line
for j in range(len(line_list)):
    l_list.extend([re.sub(r, '', line_list[j]).strip()])  # detele 0-9
print(l_list)

for k in range(len(l_list)):
    term_score_dict[int(num_list[k][0])] = getScore(l_list[k], term_doc_pos_dict)

term_score_dict

[['1', 'income', 'tax', 'reduction'], ['2', 'peace', 'in', 'the', 'Middle', 'East'], ['3', 'unemployment', 'rate', 'in', 'UK'], ['4', 'industry', 'in', 'scotland'], ['5', 'the', 'industries', 'of', 'computers'], ['6', 'Microsoft', 'Windows'], ['7', 'stock', 'market', 'in', 'Japan'], ['8', 'the', 'education', 'with', 'computers'], ['9', 'health', 'industry'], ['10', 'campaigns', 'of', 'political', 'parties']]
['income tax reduction', 'peace in the Middle East', 'unemployment rate in UK', 'industry in scotland', 'the industries of computers', 'Microsoft Windows', 'stock market in Japan', 'the education with computers', 'health industry', 'campaigns of political parties']


{1: [[65, '4.8040'],
  [3533, '4.7264'],
  [3562, '3.5454'],
  [3608, '3.4910'],
  [141, '3.3262'],
  [361, '3.3262'],
  [92, '3.2311'],
  [3829, '3.1818'],
  [3420, '3.1273'],
  [3734, '3.0561'],
  [3387, '2.9626'],
  [3599, '2.9626'],
  [3529, '2.9569'],
  [3405, '2.9026'],
  [3706, '2.7539'],
  [3403, '2.6467'],
  [125, '2.5980'],
  [3417, '2.5980'],
  [3596, '2.5541'],
  [177, '2.5185'],
  [3503, '2.5024'],
  [282, '2.4832'],
  [3818, '2.4728'],
  [163, '2.4116'],
  [3708, '2.3733'],
  [3699, '2.3216'],
  [3495, '2.2833'],
  [3710, '2.2833'],
  [184, '2.2832'],
  [3838, '2.2832'],
  [3519, '2.2450'],
  [3817, '2.2450'],
  [3345, '2.2345'],
  [270, '2.1550'],
  [3407, '2.1550'],
  [3680, '2.1550'],
  [3890, '2.1550'],
  [314, '2.1025'],
  [3535, '2.1025'],
  [3606, '2.0641'],
  [3327, '1.9742'],
  [3441, '1.9742'],
  [3590, '1.9742'],
  [3705, '1.9742'],
  [3834, '1.9742'],
  [33, '1.9358'],
  [170, '1.9358'],
  [268, '1.9358'],
  [3337, '1.9358'],
  [3622, '1.9358'],
  [3775, '1.93

# Output rank txt

In [31]:
# Output to output_rank.txt
doc = open('output_rank.txt', 'w')
for key, value in term_score_dict.items():
    if (len(value) > 10):
        for i in range(10):
            print(str(key) + "," + str(value[i][0]) + "," + str(value[i][1]), file=doc)
    else:
        for i in range(len(value)):
            print(str(key) + "," + str(value[i][0]) + "," + str(value[i][1]), file=doc)
doc.close()

# 6 Input Query txt & Output to output_boolean.txt

In [16]:
# Input Query txt
with open('queries.lab2.txt', 'r') as f:
    line_list = f.readlines()
f.close()

query_dict = {}
r = """[q]"""

for i in range(len(line_list)):  
    line_list[i] = line_list[i].split(":")  # split  
for j in range(len(line_list)):
    line_list[j][0] = re.sub(r, '', line_list[j][0])  # detele q
    line_list[j][1] = line_list[j][1].strip()  # delete \n
    query_dict[line_list[j][0]] = line_list[j][1]
    
# query_dict

In [17]:
# get results.boolean dict
# boolean_dict = {int1: list[int], int2: list[int]}
boolean_dict = {}

for i in range(len(query_dict)):
    s = str(query_dict[str(i+1)])
    boolean_dict[i+1] = search(s, term_doc_pos_dict, term_position_dict)
    
# boolean_dict

In [71]:
# Output to output_boolean.txt
doc = open('output_boolean.txt', 'w')
for key, value in boolean_dict.items():
    for i in range(len(value)):
        print(str(key) + "," + str(value[i]), file=doc)
doc.close()